In [12]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import spacy
from sklearn.metrics.pairwise import linear_kernel 
import pandas as pd

In [2]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download('rslp')
nltk.download('floresta')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package floresta to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package floresta is already up-to-date!


True

### Carregando dados

In [3]:
df = pd.read_json("noticias_negocios.json")
df

,title,description,link,author,inserted_at,id,created_at,updated_at
0,Reino Unido trabalha em plano de apoio a empre...,Ministro das Finanças Jeremy Hunt reconheceu q...,https://www.infomoney.com.br/negocios/reino-un...,Estadão Conteúdo,2023-03-12 19:35:00,1308,2023-03-29 12:40:26.458209,2023-03-29 12:40:26.458209
1,UE aprova fim de vendas de veículos a combustã...,"A Comissão Europeia prometeu, no entanto, cria...",https://www.infomoney.com.br/negocios/ue-aprov...,Reuters,2023-03-29 10:26:17,83,2023-03-29 12:04:58.615525,2023-03-29 12:04:58.615525
2,"CFO da Mills: ""Estamos cautelosos, mas de olho...","Ao InfoMoney, Caroline Pepe Leonard falou sobr...",https://www.infomoney.com.br/negocios/cfo-da-m...,andersonfigoes,2023-03-29 10:00:00,86,2023-03-29 12:05:03.665450,2023-03-29 12:05:03.665450
3,Nvidia divulga novas pesquisas sobre uso de IA...,O artigo visa melhorar uma publicação de 2021 ...,https://www.infomoney.com.br/negocios/nvidia-d...,Reuters,2023-03-29 09:20:50,87,2023-03-29 12:05:04.872418,2023-03-29 12:05:04.872418
4,Amaro pede recuperação extrajudicial com dívid...,O plano envolve pagamentos somente de credores...,https://www.infomoney.com.br/negocios/amaro-pe...,Equipe InfoMoney,2023-03-28 22:37:00,101,2023-03-29 12:05:23.036939,2023-03-29 12:05:23.036939
...,...,...,...,...,...,...,...,...
891,"""Estamos dispostos a estudar e tentar viabiliz...","Ao InfoMoney, Alex Malfitani também falou sobr...",https://www.infomoney.com.br/negocios/estamos-...,andersonfigoes,2023-03-16 16:32:25,9484,2023-03-29 22:09:27.171076,2023-03-29 22:09:27.171076
892,CSN (CSNA3) faz acordo para comprar parte de t...,O valor do negócio a ser acertado pela CSN com...,https://www.infomoney.com.br/negocios/csn-csna...,Reuters,2023-03-16 15:00:39,9493,2023-03-29 22:09:43.182559,2023-03-29 22:09:43.182559
893,Azar e sorte: quebra do SVB vira oportunidade ...,"Com lacuna deixada pelo banco norte-americano,...",https://www.infomoney.com.br/negocios/azar-e-s...,Wesley Santana,2023-03-16 14:15:38,9497,2023-03-29 22:11:40.056045,2023-03-29 22:11:40.056045
894,Ex-CEO da Americanas chega para depor na CVM,A inconsistência contábil foi tornada pública ...,https://www.infomoney.com.br/negocios/ex-ceo-d...,Estadão Conteúdo,2023-03-16 13:31:00,9502,2023-03-29 22:11:49.581078,2023-03-29 22:11:49.581078


In [4]:
df = df.set_index("id")
df.drop(columns=["created_at", "updated_at", "link"], inplace=True)

In [5]:
df

,title,description,author,inserted_at
id,,,,
1308,Reino Unido trabalha em plano de apoio a empre...,Ministro das Finanças Jeremy Hunt reconheceu q...,Estadão Conteúdo,2023-03-12 19:35:00
83,UE aprova fim de vendas de veículos a combustã...,"A Comissão Europeia prometeu, no entanto, cria...",Reuters,2023-03-29 10:26:17
86,"CFO da Mills: ""Estamos cautelosos, mas de olho...","Ao InfoMoney, Caroline Pepe Leonard falou sobr...",andersonfigoes,2023-03-29 10:00:00
87,Nvidia divulga novas pesquisas sobre uso de IA...,O artigo visa melhorar uma publicação de 2021 ...,Reuters,2023-03-29 09:20:50
101,Amaro pede recuperação extrajudicial com dívid...,O plano envolve pagamentos somente de credores...,Equipe InfoMoney,2023-03-28 22:37:00
...,...,...,...,...
9484,"""Estamos dispostos a estudar e tentar viabiliz...","Ao InfoMoney, Alex Malfitani também falou sobr...",andersonfigoes,2023-03-16 16:32:25
9493,CSN (CSNA3) faz acordo para comprar parte de t...,O valor do negócio a ser acertado pela CSN com...,Reuters,2023-03-16 15:00:39
9497,Azar e sorte: quebra do SVB vira oportunidade ...,"Com lacuna deixada pelo banco norte-americano,...",Wesley Santana,2023-03-16 14:15:38


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896 entries, 1308 to 9509
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   title        896 non-null    object        
 1   description  896 non-null    object        
 2   author       896 non-null    object        
 3   inserted_at  896 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 35.0+ KB


## Stopwords

In [7]:
stopwords = nltk.corpus.stopwords.words("portuguese")
stopwords

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

## Removendo stopwords e lematizando

In [8]:
nlp = spacy.load('pt_core_news_sm')

def lematize(text):
    doc = nlp(text)

    lemmas = []
    for word in doc:
        if word.pos_ == 'VERB':
            lemma = word.lemma_
        else:
            lemma = word.text

        lemmas.append(lemma)

    return lemmas

In [9]:
def remove_stop_words(column):
    new_column = []
    for text in column:
        splited_text = text.split(" ")
        new_text = []
        for word in splited_text:
            if word.lower() not in stopwords:
                new_text.append(word.lower())
        
        text_joined = " ".join(new_text)
        text_lematized = lematize(text_joined)

        new_column.append(" ".join(text_lematized))

    return new_column

## Lematizando

In [10]:
df["title"] = remove_stop_words(df["title"])
df["description"] = remove_stop_words(df["description"])

In [11]:
df

,title,description,author,inserted_at
id,,,,
1308,reino unir trabalha plano apoio empresas afeta...,ministro finanças jeremy hunt reconhecer episó...,Estadão Conteúdo,2023-03-12 19:35:00
83,ue aprovar fim vendas veículos combustãor 2035...,"comissão europeia prometer , entanto , criar r...",Reuters,2023-03-29 10:26:17
86,"cfo mills : "" estamos cautelosos , olho oportu...","infomoney , caroline pepe leonard falar sobre ...",andersonfigoes,2023-03-29 10:00:00
87,nvidia divulgar novas pesquisas sobre uso ia m...,"artigo visar melhorar publicação 2021 google ,...",Reuters,2023-03-29 09:20:50
101,amaro pedir recuperação extrajudicial dívidas ...,plano envolver pagamentos somente credores qui...,Equipe InfoMoney,2023-03-28 22:37:00
...,...,...,...,...
9484,""" estamos dispostos estudar tentar viabilizar ...","infomoney , alex malfitani falar sobre renegoc...",andersonfigoes,2023-03-16 16:32:25
9493,csn ( csna3 ) fazer acordo comprar parte terre...,valor negócio acertar csn josé desenvolvimento...,Reuters,2023-03-16 15:00:39
9497,azar sorte : quebra svb vira oportunidade negó...,"lacuna deixar banco norte-americano , fintechs...",Wesley Santana,2023-03-16 14:15:38


## Categorizando

In [14]:
def join_title_and_description(titles, descriptions):

    joined_text = []

    for title, description in zip(titles, descriptions):
        joined_text.append(f"{title}. {description}")

    return joined_text

### Treinando TFidF

In [17]:
texts = join_title_and_description(df["title"], df["description"])

vectorizer = TfidfVectorizer()
trained_vector = vectorizer.fit_transform(texts)

### Treinando KMeans

In [36]:
kmeans = KMeans(n_clusters=25)
kmeans.fit(trained_vector)

c:\Users\user\Documents\furb\pln\ETL-NEWS\venv\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=25)

In [37]:
categories = kmeans.predict(trained_vector)

In [38]:
texts_per_categy = {}
for i in range(len(texts)):
    if not texts_per_categy.get(categories[i]):
        texts_per_categy[categories[i]] = 0

    texts_per_categy[categories[i]] += 1

print(texts_per_categy)

{6: 49, 11: 34, 10: 28, 2: 38, 4: 55, 21: 42, 14: 23, 7: 43, 1: 46, 20: 32, 16: 35, 17: 50, 24: 25, 15: 27, 9: 32, 13: 38, 12: 34, 23: 54, 22: 32, 5: 37, 8: 39, 0: 17, 3: 30, 18: 22, 19: 34}


In [39]:
for i in range(15):
    print(f'Frase: {texts[i]} - Categoria: {categories[i]}')

Frase: reino unir trabalha plano apoio empresas afetar quebra svb. ministro finanças jeremy hunt reconhecer episódio representar " risco sério " companhias " mais promissoras " setor tecnológico - Categoria: 6
Frase: ue aprovar fim vendas veículos combustãor 2035 ; montadoras reagir. comissão europeia prometer , entanto , criar rota legal vendar carros novos funcionem " e-fuels " após 2035 ; entendar - Categoria: 11
Frase: cfo mills : " estamos cautelosos , olho oportunidades aquisições 2023 ". infomoney , caroline pepe leonard falar sobre captações crescimento prever ano , abertura filiais potencial linha amarela - Categoria: 10
Frase: nvidia divulgar novas pesquisas sobre uso ia melhorar designs chips. artigo visar melhorar publicação 2021 google , cujas descobertas tarde tornar objeto controvérsias - Categoria: 2
Frase: amaro pedir recuperação extrajudicial dívidas somar r$ 244,5 milhões. plano envolver pagamentos somente credores quirografários , possuir direito real garantia , ade